# Estimating a PIM

This report is a summary of the code found in 1_Scripts. It contains two elements:

1. We will try to estimate beta parameters of the normal linear model step by step and compare this with the estimation done through the PIM package. 
2. We will compare the time needed to estimate the parameters for $N = 25$ either through the *pim* package, or by calculating it manually.

In [ ]:
library(nleqslv)
library(pim)
library(dplyr)

Define global variables.

In [1]:
n <- 25
u <- 1
alpha <- 1
sigma <- 1

Now generate the predictor and data

In [2]:
X <- runif(n = n, min = 0.1, max = u)
Y <- alpha*X + rnorm(n = n, mean = 0, sd = sigma)
TrueBeta <- alpha/(sqrt(2) * sigma)

First fit the PIM model, using the PIM package

In [4]:
PIMfit <- pim(formula = Y ~ X, data = data.frame(Y = Y, X = X), link = 'probit', model = 'difference')
summary(PIMfit)
PIMfit@coef

pim.summary of following model : 
 Y ~ X
Type:  difference 
Link:  probit 


  Estimate Std. Error z value Pr(>|z|)  
X   1.0883     0.4707   2.312   0.0208 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Null hypothesis: b = 0 

X 
1.088254

Now try it manually. 
We will

* Create a set of pseudo-observations
* Calculate Z
* Calculate Z times beta using an initial guess of beta
* Formulate the estimating equation
* Solve the latter using a function and the *nleqslv* function.

In [6]:
# Manually
# Step one: create the set of pseudo-observations
IndX <- X %>% data.frame('X' = .) %>% mutate(index = 1:length(X))
PseudoObs <- data.frame(expand.grid('Y' = Y,'Yprime' = Y),
                expand.grid('IY' = 1:length(Y),'IYprime' = 1:length(Y))) %>%
                rowwise() %>% mutate(X = IndX[which(IndX$index == IY),'X'],
                Xprime = IndX[which(IndX$index == IYprime),'X']) %>%
                filter(IY != IYprime) %>% select(-IY,-IYprime) %>%
                mutate(PO = ifelse(Y < Yprime,1,
                                   ifelse(Y == Yprime,0.5,0)))

# Filter only those observations I(Y <= Yprime)
IndPseudObs <- PseudoObs %>% filter(PO > 0)

# Calculate Z
IndPseudObs <- mutate(IndPseudObs, Z = Xprime - X)

# Initial beta value for one predictor: one number
beta <- matrix(0, ncol = 1, nrow = 1)

# Z vector with length number of pseudo obs
Z <- IndPseudObs %>% select(Z) %>% as.matrix(., ncol = 1)

# Z times beta: matrix of [nPseudo x 1]
Zbeta <- c(Z%*%beta) %>% as.matrix(., ncol = 1)

# Pseudo observations
PO <- IndPseudObs %>% select(PO)

# Estimating equation:
# colSums(Z*dnorm(Zbeta) * (PO - pnorm(Zbeta) / c(pnorm(Zbeta)*(1-pnorm(Zbeta)))))

# To solve, have estimating equation in a separate function
PIM.ScoreFunction <- function(Z, PO){
  U.func <- function(beta, Z, PO){
    Zbeta <- c(Z%*%beta)
    colSums(Z*dnorm(Zbeta)*(PO - pnorm(Zbeta))/c(pnorm(Zbeta)*(1-pnorm(Zbeta))))
  }
  return(U.func)
}

coef <- nleqslv(x = rep(0,ncol(Z)), PIM.ScoreFunction(Z = Z, PO = PO), Z=Z, PO=PO)$x

Now compare this result with the one obtained through the package. 

In [7]:
data.frame('PIM' = PIMfit@coef, 'Manual' = coef)

,PIM,Manual
X,1.088254,1.088254


We get the same parameters!

# Creating pseudo-observations

Note that in the code above, it is not really clear how the pseudo-observations are calculated. To solve this problem, we here define a new function, with some comments.

In [9]:
CreatePO <- function(Y, X){
  if(class(X)!='data.frame') X <- data.frame('X' = X)
  # We will use indicators for X to get their corresponding Y values later on
  IndX <- X %>% mutate(index = 1:length(X))
      
  # First create all combinations between the observed Y
  Yvalues <- data.frame(expand.grid('Y' = Y,'Yprime' = Y),
                        expand.grid('IY' = 1:length(Y),'IYprime' = 1:length(Y)))
  # Now create P0 and select the I:= Y <= YPrime
  POdoubles <-  Yvalues %>%  mutate(PO = ifelse(Y < Yprime,1,
            ifelse(Y == Yprime,0.5,0))) %>% filter(PO > 0)
  # Notice that we compare Y with itself in the data frame. We delete these duplicates.
  POsingles <- POdoubles %>% filter(IY != IYprime)

  # Now we need to add the X and Xprime variables that correspond to the observed Y or Yprime
      # We use the indicators for Y and X to do this.
  PO <- POsingles %>% rowwise() %>% mutate(X = IndX[which(IndX$index == IY),'X'],
            Xprime = IndX[which(IndX$index == IYprime),'X'])

  return(PO %>% select(-IY,-IYprime))
}

# Compare speed
In this section, I will run both approaches 5000 times and compare the speed to do this. We do this to see if we can get speed advantages in the calculations when we do not load in the complete *pim* library.

In [10]:
nsim <- 5000

### Package

In [11]:
t1 <- Sys.time()
set.seed(1990)
for(i in 1:nsim){
  # First generate the predictor values.
  # This is equally spaced between [0,u]
  X <- runif(n = n, min = 0.1, max = u)

  # Generate data
  Y <- alpha*X + rnorm(n = n, mean = 0, sd = sigma)

  # PIM package beta parameter
  speedTest <- pim(formula = Y ~ X, link = 'probit', model = 'difference')@coef
}
PackageSpeed <- Sys.time() - t1

### Manual

In [12]:
t1 <- Sys.time()
set.seed(1990)
for(i in 1:nsim){
  # First generate the predictor values.
  # This is equally spaced between [0,u]
  X <- runif(n = n, min = 0.1, max = u)

  # Generate data
  Y <- alpha*X + rnorm(n = n, mean = 0, sd = sigma)
  # Create the pseudo-observations
  Observations <- CreatePO(Y = Y, X = X)
  # Calculate Z
  Z <- mutate(Observations, Z = Xprime - X) %>% select(Z) %>% as.matrix(., ncol = 1)
  # Pseudo observations
  PO <- Observations %>% select(PO)

  # Estimation
  speedTest <- nleqslv(x = rep(0,ncol(Z)), PIM.ScoreFunction(Z = Z, PO = PO), Z = Z, PO = PO)$x
}
ManualSpeed <- Sys.time() - t1

Results:

In [13]:
data.frame('Package speed' = PackageSpeed,
           'Manual speed' = ManualSpeed,
           'Number of simulations' = nsim,
           'Sample size' = n)

Package.speed,Manual.speed,Number.of.simulations,Sample.size
16.84069 secs,3.445714 mins,5000,25


> For now, better to use the package!